# 01 — Data Collection (Kaggle Datasets)

**Objective:** Load real Reddit post data from Kaggle datasets, clean and standardize columns, store in SQLite, and prepare for NLP analysis.

**Data Sources:**
- `data_science.csv` — 500K+ posts/comments from r/datascience (Kaggle)
- `reddit_posts.csv` — Multi-subreddit Reddit posts dataset (Kaggle)

**Note:** These datasets replace the Reddit API scraper. The rest of the pipeline (preprocessing, sentiment, topics, dashboard) works identically with this data.

In [1]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
from datetime import datetime
import sqlite3
import os

from src.utils import load_config, get_db_connection, init_database

config = load_config('../config/config.yaml')
print("Project config loaded")
print("Target subreddits:", config['reddit']['subreddits'])

2026-02-25 15:40:53.930 | INFO     | src.utils:load_config:45 - Configuration loaded from ..\config\config.yaml


Project config loaded
Target subreddits: ['technology', 'datascience', 'MachineLearning', 'cscareerquestions', 'programming']


## 1.1 Load the Data Science Dataset

This dataset contains posts AND comments from r/datascience. We filter to posts only (comments have title="Comment").

In [2]:
# Load data_science.csv
# Adjust the path to wherever your CSV is located
ds_path = '../data/raw/data_science.csv'

if os.path.exists(ds_path):
    df_ds = pd.read_csv(ds_path, low_memory=False)
    print(f"Loaded {len(df_ds):,} rows from data_science.csv")
    
    # Filter out comments — keep only actual posts
    df_ds = df_ds[df_ds['title'] != 'Comment'].copy()
    print(f"After filtering comments: {len(df_ds):,} actual posts")
    
    # Standardize columns to match our pipeline
    df_ds = df_ds.rename(columns={
        'comms_num': 'num_comments',
        'body': 'selftext',
        'created': 'created_utc_raw',
    })
    
    # Add subreddit column
    df_ds['subreddit'] = 'datascience'
    
    # Parse timestamps
    df_ds['created_utc'] = pd.to_datetime(df_ds['created_utc_raw'], unit='s', utc=True, errors='coerce')
    
    # Fill missing columns
    df_ds['upvote_ratio'] = 0.85  # Default estimate
    df_ds['author'] = 'kaggle_user'
    
    # Select standard columns
    ds_cols = ['id', 'subreddit', 'title', 'selftext', 'score', 'num_comments', 'created_utc', 'upvote_ratio', 'author']
    df_ds = df_ds[[c for c in ds_cols if c in df_ds.columns]].copy()
    
    print(f"\nDate range: {df_ds['created_utc'].min()} to {df_ds['created_utc'].max()}")
    print(f"Avg score: {df_ds['score'].mean():.1f}")
    print(f"Null selftext: {df_ds['selftext'].isna().sum()}")
    df_ds.head()
else:
    print(f"File not found: {ds_path}")
    print("Make sure data_science.csv is in data/raw/")
    df_ds = pd.DataFrame()

Loaded 21,095 rows from data_science.csv
After filtering comments: 1,870 actual posts

Date range: 2021-12-08 22:45:24+00:00 to 2022-04-22 08:57:43+00:00
Avg score: 51.6
Null selftext: 201


## 1.2 Load the Reddit Posts Dataset

This is the multi-subreddit dataset. We filter to tech-related subreddits.

In [3]:
# Load the larger reddit posts dataset
# Try common filenames — adjust if yours is different
possible_paths = [
    '../data/raw/reddit_database.csv',
    '../data/raw/sample.xlsx',
    '../data/raw/Reddit Top Posts.csv',
    '../data/raw/reddit_top_posts.csv',
]

df_posts = pd.DataFrame()

for path in possible_paths:
    if os.path.exists(path):
        print(f"Found: {path}")
        if path.endswith('.xlsx'):
            df_posts = pd.read_excel(path)
        else:
            df_posts = pd.read_csv(path, low_memory=False)
        print(f"Loaded {len(df_posts):,} rows")
        break

if len(df_posts) > 0:
    print(f"Columns: {list(df_posts.columns)}")
    print(f"Subreddits: {df_posts['subreddit'].value_counts().head(20)}")
    
    # Standardize columns
    rename_map = {
        'created_date': 'created_utc',
        'created_timestamp': 'created_utc_unix',
        'num_comments': 'num_comments',
        'post': 'selftext',
        'full_link': 'url',
    }
    df_posts = df_posts.rename(columns={k: v for k, v in rename_map.items() if k in df_posts.columns})
    
    # Parse timestamps
    if 'created_utc' in df_posts.columns:
        df_posts['created_utc'] = pd.to_datetime(df_posts['created_utc'], utc=True, errors='coerce')
    elif 'created_utc_unix' in df_posts.columns:
        df_posts['created_utc'] = pd.to_datetime(df_posts['created_utc_unix'], unit='s', utc=True, errors='coerce')
    
    # Fill missing columns
    if 'upvote_ratio' not in df_posts.columns:
        df_posts['upvote_ratio'] = 0.85
    if 'author' not in df_posts.columns:
        df_posts['author'] = 'kaggle_user'
    
    # Filter to tech-related subreddits if multiple exist
    tech_subs = ['technology', 'datascience', 'MachineLearning', 'cscareerquestions', 
                 'programming', 'analytics', 'dataengineering', 'learnpython',
                 'Python', 'artificial', 'computerscience', 'coding']
    
    available_tech = [s for s in tech_subs if s in df_posts['subreddit'].unique()]
    if available_tech:
        df_posts = df_posts[df_posts['subreddit'].isin(available_tech)]
        print(f"\nFiltered to tech subreddits: {available_tech}")
    
    print(f"Posts after filtering: {len(df_posts):,}")
    print(f"Date range: {df_posts['created_utc'].min()} to {df_posts['created_utc'].max()}")
else:
    print("No reddit posts file found. Listing files in data/raw/:")
    for f in os.listdir('../data/raw/'):
        print(f"  {f}")
    print("\nRename your file or update the path above.")

Found: ../data/raw/reddit_database.csv
Loaded 545,427 rows
Columns: ['created_date', 'created_timestamp', 'subreddit', 'title', 'id', 'author', 'author_created_utc', 'full_link', 'score', 'num_comments', 'num_crossposts', 'subreddit_subscribers', 'post']
Subreddits: subreddit
MachineLearning         121385
datascience              67743
statistics               61382
learnmachinelearning     44217
computerscience          43780
AskStatistics            34472
artificial               34292
analytics                17008
datasets                 16914
deeplearning             16159
rstats                   15684
computervision           15655
DataScienceJobs          14610
MLQuestions              13462
dataengineering          11861
data                      8440
dataanalysis              5897
datascienceproject        1839
kaggle                     627
Name: count, dtype: int64

Filtered to tech subreddits: ['datascience', 'MachineLearning', 'analytics', 'dataengineering', 'artificial

## 1.3 Combine Datasets

In [4]:
# Standardize both dataframes to common columns
standard_cols = ['id', 'subreddit', 'title', 'selftext', 'score', 'num_comments', 
                 'created_utc', 'upvote_ratio', 'author']

frames = []

if len(df_ds) > 0:
    # Ensure all standard columns exist
    for col in standard_cols:
        if col not in df_ds.columns:
            df_ds[col] = None
    frames.append(df_ds[standard_cols])
    print(f"Data Science posts: {len(df_ds):,}")

if len(df_posts) > 0:
    for col in standard_cols:
        if col not in df_posts.columns:
            df_posts[col] = None
    frames.append(df_posts[standard_cols])
    print(f"Reddit Posts dataset: {len(df_posts):,}")

if frames:
    df_raw = pd.concat(frames, ignore_index=True)
else:
    print("No data loaded! Using synthetic data as fallback...")
    from src.utils import generate_sample_data
    df_raw = generate_sample_data(n_posts=5000, seed=42)

# Clean up
df_raw['selftext'] = df_raw['selftext'].fillna('')
df_raw['title'] = df_raw['title'].fillna('')
df_raw['score'] = pd.to_numeric(df_raw['score'], errors='coerce').fillna(0).astype(int)
df_raw['num_comments'] = pd.to_numeric(df_raw['num_comments'], errors='coerce').fillna(0).astype(int)
df_raw = df_raw.dropna(subset=['created_utc'])
df_raw = df_raw.drop_duplicates(subset=['id'], keep='first')

print(f"\n{'='*50}")
print(f"COMBINED DATASET")
print(f"{'='*50}")
print(f"Total posts: {len(df_raw):,}")
print(f"Subreddits: {df_raw['subreddit'].nunique()}")
print(f"Date range: {df_raw['created_utc'].min().date()} to {df_raw['created_utc'].max().date()}")
print(f"\nSubreddit distribution:")
print(df_raw['subreddit'].value_counts())
print(f"\nScore stats:")
print(df_raw['score'].describe().round(1))
df_raw.head()

Data Science posts: 1,870
Reddit Posts dataset: 296,069

COMBINED DATASET
Total posts: 296,192
Subreddits: 6
Date range: 2008-06-23 to 2022-05-08

Subreddit distribution:
subreddit
MachineLearning    121385
datascience         67866
computerscience     43780
artificial          34292
analytics           17008
dataengineering     11861
Name: count, dtype: int64

Score stats:
count    296192.0
mean          5.2
std          34.5
min           0.0
25%           1.0
50%           1.0
75%           1.0
max        8331.0
Name: score, dtype: float64


,id,subreddit,title,selftext,score,num_comments,created_utc,upvote_ratio,author
0,swvi7j,datascience,STEM Career Change,I’m currently working as a field biologist for...,5,6,2022-02-20 07:17:13+00:00,0.85,kaggle_user
1,rc6wjp,datascience,Beast practices or frameworks for defining suc...,I'd love to learn what framework folks use for...,9,2,2021-12-09 02:00:48+00:00,0.85,kaggle_user
2,rc5knh,datascience,Data Scientist VS Product/Project manager role,"Hey everyone, I am currently a Sr. Analyst and...",86,53,2021-12-09 00:52:29+00:00,0.85,kaggle_user
3,rc324k,datascience,Is exact position title important?,I am in the data science team and my current p...,8,21,2021-12-08 22:45:24+00:00,0.85,kaggle_user
4,sxal34,datascience,Has anybody ever tried to create a model to pr...,"Data of past matches, goals, assists, home tea...",0,2,2022-02-20 20:30:13+00:00,0.85,kaggle_user


## 1.4 Store in SQLite Database

In [5]:
# Save to SQLite
conn = get_db_connection('../data/raw/reddit_posts.db')
init_database(conn)

inserted = 0
cursor = conn.cursor()
for _, row in df_raw.iterrows():
    try:
        cursor.execute(
            "INSERT OR IGNORE INTO posts (id, subreddit, title, selftext, score, num_comments, created_utc, upvote_ratio, author) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)",
            (str(row['id']), row['subreddit'], row['title'], row['selftext'],
             int(row['score']), int(row['num_comments']), str(row['created_utc']),
             float(row.get('upvote_ratio', 0.85)), str(row.get('author', 'unknown')))
        )
        inserted += cursor.rowcount
    except Exception as e:
        pass
conn.commit()
print(f"Inserted {inserted:,} posts into database")

# Verify
df_check = pd.read_sql("SELECT COUNT(*) as n, MIN(created_utc) as earliest, MAX(created_utc) as latest FROM posts", conn)
print(df_check)
conn.close()

2026-02-25 15:40:58.375 | INFO     | src.utils:get_db_connection:83 - Connected to database: ../data/raw/reddit_posts.db
2026-02-25 15:40:58.376 | INFO     | src.utils:init_database:118 - Database schema initialized


Inserted 0 posts into database
        n                   earliest                               latest
0  301192  2008-06-23 18:50:07+00:00  2025-02-27 22:57:03.197627892+00:00


## 1.5 Load Events Timeline

In [6]:
events = pd.read_csv('../data/external/tech_events_timeline.csv')
print(f"Tech events timeline: {len(events)} events")
events.head(10)

Tech events timeline: 19 events


,date,event,company,category,impact
0,2024-06-10,Apple WWDC 2024 — Apple Intelligence announced,Apple,product_launch,high
1,2024-06-18,NVIDIA becomes most valuable company globally,NVIDIA,milestone,high
2,2024-07-18,CrowdStrike outage crashes millions of Windows...,Microsoft,incident,high
3,2024-08-05,Google found guilty of monopoly in DOJ antitru...,Google,legal,high
4,2024-09-09,Apple iPhone 16 launch event,Apple,product_launch,medium
5,2024-09-12,OpenAI releases o1 reasoning model,OpenAI,product_launch,high
6,2024-09-24,Meta Connect 2024 — Quest 3S and Orion AR glasses,Meta,product_launch,medium
7,2024-10-01,Google Gemini app launches across 40 languages,Google,product_launch,medium
8,2024-10-28,Microsoft reports Q1 FY25 earnings — cloud gro...,Microsoft,earnings,medium
9,2024-11-05,US Presidential Election Day,NaN,political,high


In [7]:
# Save combined raw data for next notebook
df_raw.to_parquet('../data/processed/posts_raw_combined.parquet', index=False)
print(f"Saved {len(df_raw):,} posts to data/processed/posts_raw_combined.parquet")

Saved 296,192 posts to data/processed/posts_raw_combined.parquet


## Summary

Successfully loaded real Reddit data from Kaggle datasets.

| Metric | Value |
|--------|-------|
| Data sources | Kaggle (no API needed) |
| Storage | SQLite + Parquet |

**Next:** 02_eda_and_preprocessing.ipynb